In [1]:
import numpy as np
import pandas as pd
import keras
import cv2
import matplotlib.pyplot as plt
from spp.SpatialPyramidPooling import SpatialPyramidPooling
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input, Flatten, AveragePooling2D, Dropout
from keras.models import Model
from keras.activations import relu
import keras
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
data = pd.read_csv("./training-data/train.csv", names = ['image','label'])
data.shape

(554656, 2)

In [3]:
data = data.sample(n=3000, random_state=14)

In [4]:
print(len(np.unique(data.label)), data.shape)

43 (3000, 2)


In [5]:
classes = len(np.unique(data.label))

In [6]:
data_labels = keras.utils.to_categorical(data.label, num_classes=43)
trainX, testX, trainY, testY = train_test_split(data, data_labels, test_size = 0.1, random_state = 42)
trainX = trainX.reset_index().drop('index', axis=1)
testX = testX.reset_index().drop('index', axis=1)

In [7]:
print(trainX.shape, testX.shape, trainY.shape, testY.shape)

(2700, 2) (300, 2) (2700, 43) (300, 43)


### Data Generator

In [8]:
def data_gen(gen_data, batch_size):
    idx_start = 0
    if gen_data == 'train':
        data = trainX
        data_labels = trainY
    elif gen_data == 'test':
        data = testX
        data_labels = testY
    
    image_batch = np.zeros((batch_size, 224,224,3))
    label_batch = np.zeros((batch_size, 43))
    
    while True:
        batch_i = 0
        while (batch_i < batch_size):
            if idx_start < data.shape[0]:
                file = './training-data/train-images/' + data.image[idx_start]
                image = cv2.imread(file)
                image = cv2.resize(image, (224,224))
                image_batch[batch_i] = image /255.0
                label_batch[batch_i] = data_labels[idx_start]
                idx_start += 1
                batch_i += 1
            else:
                idx_start = 1
                print("going again")
                
        #idx_start += batch_size
        image_batch =np.array(image_batch) 
        #print(image_batch.shape, label_batch.shape)
        yield image_batch,label_batch

In [9]:
input_shape = (224,224,3)
epochs = 2
batch_size = 32

In [10]:
base_model = keras.applications.inception_v3.InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')

In [11]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

__________________________________________________________________________________________________
mixed0 (Concatenate)            (None, 25, 25, 256)  0           activation_6[0][0]               
                                                                 activation_8[0][0]               
                                                                 activation_11[0][0]              
                                                                 activation_12[0][0]              
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 25, 25, 64)   16384       mixed0[0][0]                     
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 25, 25, 64)   192         conv2d_16[0][0]                  
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
batch_normalization_20 (BatchNo (None, 25, 25, 64)   192         conv2d_20[0][0]                  
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 25, 25, 64)   192         conv2d_22[0][0]                  
__________________________________________________________________________________________________
batch_normalization_25 (BatchNo (None, 25, 25, 96)   288         conv2d_25[0][0]                  
__________________________________________________________________________________________________
batch_normalization_26 (BatchNo (None, 25, 25, 64)   192         conv2d_26[0][0]                  
__________________________________________________________________________________________________
activation_20 (Activation)      (None, 25, 25, 64)   0           batch_normalization_20[0][0]     
__________

batch_normalization_38 (BatchNo (None, 12, 12, 128)  384         conv2d_38[0][0]                  
__________________________________________________________________________________________________
activation_33 (Activation)      (None, 12, 12, 128)  0           batch_normalization_33[0][0]     
__________________________________________________________________________________________________
activation_38 (Activation)      (None, 12, 12, 128)  0           batch_normalization_38[0][0]     
__________________________________________________________________________________________________
average_pooling2d_4 (AveragePoo (None, 12, 12, 768)  0           mixed3[0][0]                     
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 12, 12, 192)  147456      mixed3[0][0]                     
__________________________________________________________________________________________________
conv2d_34 

__________________________________________________________________________________________________
batch_normalization_41 (BatchNo (None, 12, 12, 192)  576         conv2d_41[0][0]                  
__________________________________________________________________________________________________
batch_normalization_44 (BatchNo (None, 12, 12, 192)  576         conv2d_44[0][0]                  
__________________________________________________________________________________________________
batch_normalization_49 (BatchNo (None, 12, 12, 192)  576         conv2d_49[0][0]                  
__________________________________________________________________________________________________
batch_normalization_50 (BatchNo (None, 12, 12, 192)  576         conv2d_50[0][0]                  
__________________________________________________________________________________________________
activation_41 (Activation)      (None, 12, 12, 192)  0           batch_normalization_41[0][0]     
__________

activation_60 (Activation)      (None, 12, 12, 192)  0           batch_normalization_60[0][0]     
__________________________________________________________________________________________________
mixed6 (Concatenate)            (None, 12, 12, 768)  0           activation_51[0][0]              
                                                                 activation_54[0][0]              
                                                                 activation_59[0][0]              
                                                                 activation_60[0][0]              
__________________________________________________________________________________________________
conv2d_65 (Conv2D)              (None, 12, 12, 192)  147456      mixed6[0][0]                     
__________________________________________________________________________________________________
batch_normalization_65 (BatchNo (None, 12, 12, 192)  576         conv2d_65[0][0]                  
__________

batch_normalization_74 (BatchNo (None, 12, 12, 192)  576         conv2d_74[0][0]                  
__________________________________________________________________________________________________
activation_74 (Activation)      (None, 12, 12, 192)  0           batch_normalization_74[0][0]     
__________________________________________________________________________________________________
conv2d_71 (Conv2D)              (None, 12, 12, 192)  147456      mixed7[0][0]                     
__________________________________________________________________________________________________
conv2d_75 (Conv2D)              (None, 12, 12, 192)  258048      activation_74[0][0]              
__________________________________________________________________________________________________
batch_normalization_71 (BatchNo (None, 12, 12, 192)  576         conv2d_71[0][0]                  
__________________________________________________________________________________________________
batch_norm

activation_84 (Activation)      (None, 5, 5, 384)    0           batch_normalization_84[0][0]     
__________________________________________________________________________________________________
batch_normalization_85 (BatchNo (None, 5, 5, 192)    576         conv2d_85[0][0]                  
__________________________________________________________________________________________________
activation_77 (Activation)      (None, 5, 5, 320)    0           batch_normalization_77[0][0]     
__________________________________________________________________________________________________
mixed9_0 (Concatenate)          (None, 5, 5, 768)    0           activation_79[0][0]              
                                                                 activation_80[0][0]              
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 5, 5, 768)    0           activation_83[0][0]              
          

                                                                 mixed9_1[0][0]                   
                                                                 concatenate_2[0][0]              
                                                                 activation_94[0][0]              
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
__________________________________________________________________________________________________


In [13]:
x = base_model.output
x = AveragePooling2D(pool_size=(5,5))(x)
x = Dropout(.4)(x)
x = Flatten()(x)
predictions = Dense(43, init='glorot_uniform', W_regularizer=keras.regularizers.l2(.0005), activation='sigmoid')(x)
model = Model(input=base_model.input, output=predictions)
opt = keras.optimizers.Adam(lr=1e-4)
opt = keras.optimizers.SGD(lr=.01, momentum=.9)
model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

c:\users\slayer\miniconda3\envs\tf_cuda\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(43, activation="sigmoid", kernel_initializer="glorot_uniform", kernel_regularizer=<keras.reg...)`
  """
c:\users\slayer\miniconda3\envs\tf_cuda\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [14]:
#callbacks
early_stopper = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto', baseline=None)
checkpointer = keras.callbacks.ModelCheckpoint(filepath='./model.hdf5', verbose=1, save_best_only=True)

In [15]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):

    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
        val_targ = self.model.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        #print(" — val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
        return

conf_metrics = Metrics()

In [16]:
history = model.fit_generator(data_gen('train', batch_size),
                              callbacks=[early_stopper, checkpointer],
                              validation_data= data_gen('test', batch_size),
                              validation_steps=testX.shape[0]//batch_size,
                              epochs=epochs, steps_per_epoch = trainX.shape[0]//batch_size)

Epoch 1/2
going again
84/84 [==============================] - 52s 625ms/step - loss: 3.2483 - acc: 0.2176 - val_loss: 3.3194 - val_acc: 0.1840

Epoch 00001: val_loss improved from inf to 3.31935, saving model to ./model.hdf5
Epoch 2/2
84/84 [==============================] - 34s 400ms/step - loss: 3.0747 - acc: 0.2303 - val_loss: 3.1884 - val_acc: 0.1910

Epoch 00002: val_loss improved from 3.31935 to 3.18844, saving model to ./model.hdf5


In [17]:
model.evaluate_generator(data_gen('test', batch_size), steps=10)

going again


[3.087937068939209, 0.21875]